In [1]:
#%matplotlib notebook
#%pylab qt
#from matplotlib import pyplot as plt

In [2]:
from matplotlib import animation, rc
from IPython.display import HTML
import numpy as np
rc('animation', html='jshtml')


For the purpose of checking this script alone I leave the commeted out cell bellow

In case of debug or check can uncomment line bellow

In [15]:
# %run -p cphonon.ipynb
# fig
# display(FirstBox)
# #display(Masses)
# display(Impurity)
# display(ModeNrInterBox)

Initializing values

In [8]:
#mode=ModeNrInter.value
if MyInteraction.result is None:
    Vanim=V[:,0]
    omegaAnim=omega[0]
else:
    Vanim=MyInteraction.result[2]
    omegaAnim=MyInteraction.result[1]
N=NInter.value
bc=bcInter.value
ModeNr=ModeNrInter.value
M1=M1Inter.value
M2=M2Inter.value
Mimp=MimpInter.value
Nimp=NimpInter.value
C=CInter.value
imp_enabled=imp_enabledInter.value
omegaAnim=5 if np.abs(omegaAnim)<1e-06 else omegaAnim #Otherwise it will mant to make an infinite animation

Dealing with the case of fixed boundary conditions and N=2. This solves the problem

In [9]:
try:
    len(Vanim)
except:
    Vanim=[[0,0],[0,0]]
    omegaAnim=10

28

In [10]:
Ampl=0.1*((M1+M2)/C)**(1/4) # don't know the reason for exactly ^(1/4)
Vplot=np.array(np.transpose(Vanim)*Ampl)[0]  #transpose and [0] is for some stupid reasons with arrays

oddAtoms = np.arange(1, N+1, 2)
evenAtoms = np.arange(2, N+1, 2)
allAtoms = np.arange(1, N+1)
Vodd=Vplot[::2]
Veven=Vplot[1::2]

yodd = [0]*len(Vodd)
yeven = [0]*len(Veven)

In [14]:
N//2

14

In [1]:
%%capture
#%matplotlib tk
fig2, ax3 = plt.subplots()
fig2.set_size_inches(10.0, 1.4)

ax3.set_xlim(( 0.5, N+0.5))
ax3.set_ylim((-0.05, 0.05))
#fig2.title='MyTitle'


line,=ax3.plot([], [], 'bo', markersize=10)

if M1 > M2:
        mark1 = 11
        mark2 = 6
elif M1 == M2:
        mark1 = mark2 = 6
else:
        mark1 = 6
        mark2 = 11

#Marker colors for masses
if M1==M2:
    col1, col2="blue", "blue"
else:
    col1, col2="blue", "green"
        
        
#plotlays=[2]
plotcols, markers = [col1,col2,"white","red"], [mark1,mark2,11,(np.log(Mimp/M1+1.7)*7)] #["blue","green"],[mark1,mark2] 
lines = []
for index in range(4):
    lobj = ax3.plot([],[],'o',markersize=markers[index],color=plotcols[index])[0]
    lines.append(lobj)
def init():
    for line in lines:
        line.set_data([],[])
    return lines

ax3.set(xlabel="postition/a, m", ylabel='y',title='Atomic oscilations, mode nr={}'.format(ModeNr)) # for N={}, M1={}, M2={},Minp={}.'.format(N,M1,M2,Mimp)

steptick=N//15+1
xTickArray=np.arange(0, N+2, step=steptick)
xTickArray = xTickArray.astype('float64')
xTickArray[0], xTickArray[-1]=0.5, N+0.5
ax3.set_xticks(xTickArray)

fig2.subplots_adjust(bottom=0.3,top=0.8)

NameError: name 'plt' is not defined

Note how big problems the periodic boundary conditions cause(particle can appear from the other side). Otherwise the function bellow bould have been pretty simple. Now you have to look through all the cases and change a bit the  plotting data.

It still does not deal with the case when the impurity is the last particle and it should reapear on the other side. If somebody really creates such a case then congrats, u cool!

In [12]:
def animate(t, *args):
    lines[0].set_data(oddAtoms+np.cos(omegaAnim*t*0.02*2*np.pi)*Vodd, yodd)
    lines[1].set_data(evenAtoms+np.cos(omegaAnim*t*0.02*2*np.pi)*Veven, yeven)
    if bc==0: #periodic boundary conditions (particle can appear from the other side) causes problems
        if np.cos(omegaAnim*t*0.02*2*np.pi)*Vodd[0]< -0.5:
            lines[0].set_data(np.append(oddAtoms[1:],N+1)+
                              np.cos(omegaAnim*t*0.02*2*np.pi)*np.append(Vodd[1:],Vodd[0]), yodd)
            lines[1].set_data(evenAtoms+np.cos(omegaAnim*t*0.02*2*np.pi)*Veven, yeven)
        elif (np.cos(omegaAnim*t*0.02*2*np.pi)*Vodd[-1]> 0.5) & (N%2) :
            lines[0].set_data(np.append(0,oddAtoms)+
                              np.cos(omegaAnim*t*0.02*2*np.pi)*np.append(Vodd[-1],Vodd), np.append(0,yodd)) #I don't take out the data point because it is not visible anyway
            lines[1].set_data(evenAtoms+np.cos(omegaAnim*t*0.02*2*np.pi)*Veven, yeven)
        elif (np.cos(omegaAnim*t*0.02*2*np.pi)*Veven[-1]> 0.5) & (N%2==0) :
            lines[0].set_data(oddAtoms+
                              np.cos(omegaAnim*t*0.02*2*np.pi)*Vodd, yodd) #I don't take out the data point because it is not visible anyway
            lines[1].set_data(np.append(0,evenAtoms)+np.cos(omegaAnim*t*0.02*2*np.pi)*np.append(Veven[-1],Veven), np.append(0,yeven))
    if imp_enabled == 1:
        lines[2].set_data(Nimp+np.cos(omegaAnim*t*0.02*2*np.pi)*Vplot[Nimp-1],0)
        lines[3].set_data(Nimp+np.cos(omegaAnim*t*0.02*2*np.pi)*Vplot[Nimp-1],0)
    return (lines)



anim = animation.FuncAnimation(fig2, animate, init_func=init,
                              frames=int(np.round(50/omegaAnim)), interval=4, blit=True) #frames value taken so that the film ends exatly at the end of the period

NameError: name 'fig2' is not defined

In [86]:
#%%time
#HTML(anim.to_jshtml())
#anim

In [97]:
#omegaAnim

9.125060374972142e-09

In [23]:
#fig2

In [19]:
#arr

In [ ]:
#arr[5]=0.5

In [ ]:
#float(arr)

In [ ]:
# from matplotlib import pyplot as plt
# %matplotlib tk
# %matplotlib tk
#%%time
#%pylab qt
#display(HTML(anim.to_jshtml()))
#anim

In [ ]:
#out = widgets.Output(layout={'border': '1px solid black'})
# with outp:
#     anim
#outp
#anim


In [ ]:
#outp.append_stdout('Output appended with append_stdout')
#outp.append_display_data(anim)

In [ ]:
# import pandas as pd
# import numpy as np
# df = pd.DataFrame(np.random.randn(6,4),columns=list('ABCD'))
# # Show in Jupyter
# df

# from IPython.display import HTML
# s  = '<script type="text/Javascript">'
# s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
# s += 'win.document.body.innerHTML = \'' + HTML(anim.to_jshtml()) + '\';'
# s += '</script>'

# # Show in new Window
# HTML(s)

In [ ]:
#outp.clear_output()